在计算机视觉的实际应用中，分类是远远不够的，更多的场景需要能同时探测器多个物体，不仅包括这些物体的分类，而且包括这些物体的位置和尺寸。通常使用包围盒（Bounding Box）来表示这些物体的位置和尺寸。

![img](images/chapter15/object_detection.png)

经典的使用神经网络实现目标检测的模型有：

- R-CNN
- SPPNet
- Fast R-CNN
- Faster R-CNN
- SDD
- YOLO

# 1. [R-CNN](https://arxiv.org/abs/1311.2524)

R-CNN（Region-based Convolutional Network） 模型发展到现在已经成为一个系列，从最初的 R-CNN 模型发展到 Fast R-CNN 模型，之后在 2015 年再一次突破，提出了 Faster R-CNN模型。近几年来，基于深度学习的目标检测方法发展很快，相继提出了很多精妙的思想，R-CNN 模型系列功不可没，该模型是很多算法的基础思想。

R-CNN 结合了 [Selective Search](https://www.cs.mcgill.ca/~hsyed2/selectivesearch/Object-Recognition/) 算法的建议窗口和由卷积神经网络产生的视觉丰富特征，它的性能比当时最好的检测模型在 Pascal VOC 2012 数据集上有将近 30% 的性能提升，mAP（mean average precision）指标从 40.9% 提升到了 53.3%。

R-CNN 模型的整体框架为：

- 采用 Selective Search 算法生成建议窗口（region proposals）
      每张图片大约生成 2000 个建议窗口，由于建议窗口尺寸不一，所以通过拉伸变换，形成统一的输入尺寸 227×227。

- 运用 CNN 网络进行特征提取
      将统一尺寸的建议窗图像输入 AlexNet 网络并提取 4096 维特征向量
      
- 对建议窗图像进行分类
      使用 SVM 分类器对 CNN 输出特征进行分类
      
- 包围盒回归（Bounding Box Regression）
      用 Bounding Box 回归值校正原来的建议窗口，生成预测窗口坐标

![img](images/chapter15/R-CNN.png)

R-CNN 的训练步骤分为三步：

- 在 ImageNet 数据集上进行预训练（pre-training）
![img](images/chapter15/R-CNN_pre_train.png)


- 利用成熟的权重参数在 PASCAL VOC 数据集上进行微调（fine-tune）
      将网络最后一个 1000 个神经元的全连接层改为 21 个神经元
      
![img](images/chapter15/R-CNN_fine_tune.png)
      
- 使用 CNN 提取的特征作为输入信息，为每个分类训练一个 SVM，可以将精度再提高 3.3%

![img](images/chapter15/R-CNN_train_SVM.png)

R-CNN 模型由于提出较早，相比于目前的目标检测网络模型，精确度不够高，速度也较慢，无法满足实时系统需要。但 R-CNN 让业界第一次看到了深度学习在目标检测上的力量和潜力，是深度学习在目标检测任务上迈出的重要一步。

# 2. [Fast R-CNN](https://arxiv.org/abs/1504.08083)

R-CNN 模型主要存在以下三个问题：

- 测试速度慢

      R-CNN 将一张图像分解成近 2000 个建议窗口，每个建议窗口拉伸形成的图像都会单独通过 CNN 提取特征。实际上这些建议窗口之间大量重叠，特征值之间完全可以共享，因此造成了运算能力的浪费
      
- 训练速度慢
      R-CNN 在训练时，在应用 SVM 分类之前，将 CNN 提取的特征存储在硬盘上，在硬盘上大量读写交换数据造成训练速度缓慢
      
- 训练所需空间大
      R-CNN 中独立的 SVM 分类器和回归器需要大量特征作为训练样本，需要存储上千GB的数据

R-CNN 的原作者 Ross Girshick 在吸收了 [SPPNet](https://arxiv.org/abs/1406.4729) 的优点后再一次推出了 R-CNN 的继承者 Fast R-CNN。Fast R-CNN 构思精巧，流程更为紧凑，大幅提升了目标检测的速度。与 R-CNN 相比，训练时间从 84 小时减少为 9.5 小时，测试时间从 47 秒减少为 0.32 秒，在 PASCAL VOC 2007 数据集上的准确率为 66%。

Fast R-CNN 模型整体训练框架大致为：

- 将整张图像输入至 CNN 网络，进行特征提取


- 使用 Selective Search 算法生成建议窗口，并将建议窗口映射到 CNN 的最后一层卷积特征图上


- 通过 ROI（regions of interest） Pooling 层将每个建议窗口生成固定尺寸的特征图


- 将特征图输入至全连接层网络提取 ROI 特征向量


- Softmax 层对 ROI 特征向量进行分类，回归层使用 Smooth L1 损失函数进行包围盒回归，并加权联合二者损失函数值

![img](images/chapter15/Fast_R-CNN.png)

由于用于目标检测的数据集较小，无法充分从零开始训练网络，Fast R-CNN 和 R-CNN 一样采用在预训练网络模型上微调参数的方法，即通过 ImageNet 数据集对 CNN 进行预训练，然后用 PASCAL VOC 数据集再进行微调训练。Fast R-CNN 使用了三种预训练网络模型，分别是小型网络 CaffeNet（AlexNet 修改版）、中型网络 VGG_VNN_M_1024（和 AlexNet 同样深度的 VGG 网络）和大型网络 VGG16。

Fast R-CNN 是深度学习算法在目标检测任务上的重大进步，让业界看到了深度学习算法在目标检测任务上进行 **实时处理** 的希望。